# EXP2: Implementar novas features usando o dataframe com targets atualizados

In [86]:
import pandas as pd

In [87]:
data = pd.read_pickle("/data/challenges/hackathon_bigdata_2025/processed_data/merged_transactions_with_targets.pkl")

In [88]:
data

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,marca,fabricante,descricao,gross_value,net_value,gross_profit,discount,taxes,quantidade,target
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,Michelob Ultra,AB Anheuser Busch Inc,MICHELOB ULTRA 2/12/12 CN,54.299999,49.740486,16.224486,0.000000,4.559516,2.0,Not available
1,1001371918471115422,30175.0,1120490062981954254,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 2/12/8 CN,262.350006,234.992905,62.210903,53.099998,27.357101,18.0,Not available
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 4/6/16 CN,37.950001,31.870647,8.182648,5.100000,6.079354,2.0,Not available
3,1001371918471115422,30175.0,4353552881410365573,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 2/15/12 CN,138.250000,118.302116,33.567116,2.800000,19.947886,7.0,Not available
4,1001371918471115422,30175.0,4797439216678436447,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light Lime,AB Anheuser Busch Inc,BUD LIGHT LIME 2/12/12 CN,26.750000,24.470242,7.524241,0.000000,2.279758,1.0,Not available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6166028,996297376394222790,30035.0,9024979450937837276,5,12,2022,Convenience,Non-Alcohol,Water,Off Premise,...,Essentia Water,Essentia Water LLC,ESSENTIA WATER 24/20 NR,24.600000,24.600000,8.560000,0.000000,0.000000,1.0,Not available
6166029,997907800111849739,80205.0,3516587220187345473,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Natural Ice,AB Anheuser Busch Inc,NATURAL ICE 15/25 CN,220.725006,218.615631,51.665630,24.750000,2.109375,9.0,Not available
6166030,997907800111849739,80205.0,4451855961536069898,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Budweiser,AB Anheuser Busch Inc,BUDWEISER 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available
6166031,997907800111849739,80205.0,7029448080469668619,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available


In [89]:
# Custo da mercadoria = gross_value / quantidade (para o cliente)
# Custo para da mercadoria para o pdv = net_value - gross_profit

# Valor total pago pelo cliente = gross_value; O preço original seria (gross_value + discount)
# Net_value: faturamento da venda descontando impostos
# Gross_value = net_value - custo_de_mercadoria

In [90]:
ids = data['quantidade'] > 0 # update only where quantidade > 0 to avoid division by zero
data['buyer_price_per_unit'] = 0
data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
data['pdv_cost_per_unit'] = 0
data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()

/tmp/ipykernel_470373/3990647784.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[27.14999962 14.57500034 18.97500038 ... 32.85       32.85
 37.20000076]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
/tmp/ipykernel_470373/3990647784.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[16.75800014  9.59900008 11.84399986 ... 23.35899963 23.35899963
 26.40600014]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()


In [91]:
filtered = data[data.target != "Not available"]

In [92]:
filtered['pdv_produto'] = filtered['pdv'].astype(str) + '_' + filtered['produto'].astype(str)

/tmp/ipykernel_470373/793479106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['pdv_produto'] = filtered['pdv'].astype(str) + '_' + filtered['produto'].astype(str)


In [93]:
filtered.pdv_produto.nunique()

339599

In [96]:
filtered.mes

8           1
15          1
16          1
18          1
22          1
           ..
6130825    11
6130828    11
6130829    11
6130830    11
6130833    11
Name: mes, Length: 1917729, dtype: int32

In [99]:
filtered_train = filtered[filtered.mes.isin([8, 9])]
filtered_val = filtered[filtered.mes.isin([10, 11])]

In [106]:
min_month_train = filtered_train.mes.min()
filtered_train['mes'] = filtered_train.mes - min_month_train
filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana

min_month_val = filtered_val.mes.min()
filtered_val.mes = filtered_val.mes - min_month_val
filtered_val['semana_new'] = filtered_val.mes * 5 + filtered_val.semana

/tmp/ipykernel_470373/316573106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['mes'] = filtered_train.mes - min_month_train
/tmp/ipykernel_470373/316573106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana
/tmp/ipykernel_470373/316573106.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [108]:
filtered_train.semana_new.unique(), filtered_val.semana_new.unique()

(array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32),
 array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32))

In [112]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_470373/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_470373/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [118]:
filtered_train.pdv_produto.value_counts()

pdv_produto
4440977747777161208_1860061817666925715    10
7798371321515892727_2749706558857279178    10
6798713085169683814_8728662573224341894    10
7803689727198037569_576514643033029430     10
6674004198017668031_700484830392455456     10
                                           ..
8723723113467008071_6562521907333808722     1
1001371918471115422_5645002264939911240     1
1001371918471115422_5418855670645487653     1
1001371918471115422_4974205226846185245     1
1001371918471115422_3407816598812292920     1
Name: count, Length: 166811, dtype: int64

In [119]:
def aggr_df(df, categorical_features=[
    'label', 'categoria_pdv', 'categoria'], temporal_features=['quantidade', 'discount', 'gross_value', 'net_value', 'buyer_price_per_unit', 'pdv_cost_per_unit']):
    filtered_df = df.copy()

    # Add all features and mantain target column
    first = False
    for feature in temporal_features:
        if feature not in filtered_df.columns:
            raise ValueError(f"Feature {feature} not found in dataframe columns")
        # Aggregate temporal features by pdv_produto and semana_mes using sum
        # Create pivot tables for each temporal feature
        # For example, for quantidade, create features such as "quantidade_08_1", "quantidade_08_2", ..., "quantidade_09_4"
        filtered_X_agg = filtered_df.pivot_table(index='pdv_produto', columns='semana_mes', values=feature, aggfunc='sum')
        filtered_X_agg.columns = [f'{feature}_{col}' for col in filtered_X_agg.columns]
        filtered_X_agg = filtered_X_agg.reset_index()
        if not first:
            first = True
            concat_df = filtered_X_agg.copy()
        else:
            concat_df = concat_df.merge(filtered_X_agg, on='pdv_produto', how='left')

    # Add target column
    target_agg = filtered_df.groupby('pdv_produto').agg({
        'target': 'last'
    }).reset_index()
    concat_df = concat_df.merge(target_agg, on='pdv_produto', how='left')

    #return concat_df

    # For categorical features, create features such as "label", "categoria_pdv", "categoria" using the first value found for that pdv_produto
    categorical_agg = filtered_df.groupby('pdv_produto').agg({
        feature: 'first' for feature in categorical_features
    }).reset_index()
    # Merge all features into a single dataframe
    concat_df =concat_df.merge(categorical_agg, on='pdv_produto', how='left')

    # For each categorical feature, perform one-hot encoding (label, categoria_pdv, categoria, tipos)
    #categorical_features = ['label', 'categoria_pdv' ,'categoria', 'subcategoria','tipos']
    #filtered_X_agg = pd.get_dummies(filtered_X_agg, columns=categorical_features, drop_first=True)
    concat_df = concat_df.fillna(0)

    return concat_df

In [134]:
features_train = aggr_df(filtered_train)
features_val = aggr_df(filtered_val)

/tmp/ipykernel_470373/28584920.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  concat_df = concat_df.fillna(0)
/tmp/ipykernel_470373/28584920.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  concat_df = concat_df.fillna(0)


In [130]:
(filtered_train.drop_duplicates(subset=['pdv_produto'], keep='last').sort_values('pdv_produto').reset_index().target == features_train.sort_values('pdv_produto').target).sum()

np.int64(166811)

In [137]:
features_val

,pdv_produto,quantidade_0_1,quantidade_0_2,quantidade_0_3,quantidade_0_4,quantidade_0_5,quantidade_1_1,quantidade_1_2,quantidade_1_3,quantidade_1_4,...,pdv_cost_per_unit_0_5,pdv_cost_per_unit_1_1,pdv_cost_per_unit_1_2,pdv_cost_per_unit_1_3,pdv_cost_per_unit_1_4,pdv_cost_per_unit_1_5,target,label,categoria_pdv,categoria
0,1000285336744471896_3496441975053481303,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,47.930,0.000000,0.000,0.0,0.0,1.0,Core,Theatre,Draft
1,1001371918471115422_1009179103632945474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.000,0.000000,13.384,0.0,0.0,4.0,Core,Convenience,Package
2,1001371918471115422_1029370090212151375,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000000,0.000,0.0,0.0,4.0,Core,Convenience,Package
3,1001371918471115422_1120490062981954254,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0,0.0,...,0.0,10.159,0.000000,10.159,0.0,0.0,20.0,Discontinued,Convenience,Package
4,1001371918471115422_1454838625590783593,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000000,0.000,0.0,0.0,3.0,Core,Convenience,Package
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159069,997907800111849739_7720369747531966099,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000000,0.000,0.0,0.0,1.0,Discontinued,Package/Liquor,Package
159070,997907800111849739_7967083691239671783,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,22.991,0.000000,0.000,0.0,0.0,1.0,Winter Seasonal,Package/Liquor,Package
159071,997907800111849739_802532509492578586,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000000,0.000,0.0,0.0,3.0,Core,Package/Liquor,Package
159072,997907800111849739_8581872597368912387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.000,0.000000,13.695,0.0,0.0,10.0,Core,Package/Liquor,Package
